In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgbm

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:


df_train = pd.read_csv("/home/xiaoguzai/数据/kaggle有毒评论对比/Jigsaw-toxic-comment-classification-challenge/train.csv")
# df_test = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv")
# df_test_label = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv").replace(-1,0)
#df_sub = pd.read_csv("/home/xiaoguzai/数据/kaggle有毒评论对比/Jigsaw Rate Severity of Toxic Comments/comments_to_score.csv")

In [3]:
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
# Create a score that measure how much toxic is a comment
r"""
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}
"""
r"""
cat_mtpl = {'obscene': 1, 'toxic': 2, 'threat': 0.8, 
            'insult': 0.9, 'severe_toxic': 3, 'identity_hate': 1.6}
            0.85892
"""
cat_mtpl = {'obscene': 1, 'toxic': 1, 'threat': 0.8, 
            'insult': 0.9, 'severe_toxic': 2, 'identity_hate': 1.5}
for category in cat_mtpl:
    df_train[category] = df_train[category] * cat_mtpl[category]

df_train['score'] = df_train.loc[:, 'toxic':'identity_hate'].sum(axis=1)

df_train['y'] = df_train['score']

min_len = (df_train['y'] > 0).sum()  # len of toxic comments
df_y0_undersample = df_train[df_train['y'] == 0].sample(n=min_len, random_state=201)  # take non toxic comments
df_train_new = pd.concat([df_train[df_train['y'] > 0], df_y0_undersample])  # make new df
df_train_new.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,score,y
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,2,1,0.0,0.9,0.0,4.9,4.9
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0.0,0.0,0.0,1.0,1.0


In [5]:
#df_train = pd.read_csv("/home/xiaoguzai/数据/kaggle有毒评论对比/ruddit/ruddit_with_text.csv")

In [6]:
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,score,y
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0.0,0.0,0.0,0.0,0.0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0.0,0.0,0.0,0.0,0.0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0.0,0.0,0.0,0.0,0.0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0.0,0.0,0.0,0.0,0.0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_train = df_train.rename(columns={'comment_text':'text'})
df_train = df_train.rename(columns={'offensiveness_score':'y'})
min_data = df_train.y.min()
max_data = df_train.y.max()
df_train['y'] = (df_train['y']-min_data)/(max_data-min_data)
#df_train['y'] = df_train['y']/df_train['y'].max()

In [8]:
df_train['y'].value_counts()

0.000000    143346
0.138889      5983
0.402778      3800
0.277778      1758
0.263889      1396
0.680556       989
0.611111       618
0.125000       301
0.888889       265
0.555556       158
0.472222       152
0.347222       139
0.513889       131
0.250000       115
0.791667        64
0.722222        56
0.208333        54
0.416667        41
0.486111        35
1.000000        31
0.333333        28
0.111111        22
0.375000        18
0.541667        14
0.388889        11
0.527778        11
0.750000         7
0.458333         7
0.763889         6
0.666667         4
0.625000         3
0.236111         3
0.583333         3
0.652778         1
0.736111         1
Name: y, dtype: int64

In [9]:
import re
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]
def clean(data, col):
    
    data[col] = data[col].str.replace(r"what's", "what is ")    
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    data[col] = data[col].str.replace(r"\'s", " ")

    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')    
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ')    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1')
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    data[col] = data[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    return data

In [10]:
#df_train['text'] = df_train['text'].apply(text_cleaning)
#df_train = clean(df_train,'text')

In [11]:
from sklearn.linear_model import Ridge, Lasso, BayesianRidge
from sklearn.svm import SVR
vec = TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5))
X = vec.fit_transform(df_train['text'])
model = Ridge(random_state=42,alpha=0.3)
model.fit(X, df_train['y'])

Ridge(alpha=0.3, random_state=42)

In [12]:
df_val = pd.read_csv("/home/xiaoguzai/数据/Kaggle Jigsaw Rate Severity of Toxic Comments/validation_true.csv")
df_val.head()

,Unnamed: 0,less_toxic,more_toxic
0,1,I just read what you wrote; neither of the tw...,\n\nwhy should people have to read crap posted...
1,2,"""\n\nI don't know which """"personal attacks"""" y...",\n\nwhy should people have to read crap posted...
2,5,"\nU POUR ADMIN, U UPDATE VANDAL COUNT, WHILE R...",Your relentless edit warring \n\nYour message...
3,9,\n\nFailure to follow the guidelines of WP:NO...,Against community and you don't know what devi...
4,10,\n\nGet a encyclopedia you nerds......(kanich...,"AND TO THE PARTY WHO FUCKS ME, FUCK U! I'LL BE..."


In [13]:
#df_val = pd.read_csv("/home/xiaoguzai/数据/Kaggle Jigsaw Rate Severity of Toxic Comments/validation_new.csv")
#tqdm.pandas()
#df_val['less_toxic'] = df_val['less_toxic'].apply(text_cleaning)
#df_val['more_toxic'] = df_val['more_toxic'].apply(text_cleaning)
df_val = clean(df_val,'less_toxic')
df_val = clean(df_val,'more_toxic')
X_less_toxic = vec.transform(df_val['less_toxic'])
X_more_toxic = vec.transform(df_val['more_toxic'])
p1 = model.predict(X_less_toxic)
p2 = model.predict(X_more_toxic)
print((p1 < p2).mean())

/tmp/ipykernel_20089/87036369.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r"\'ve", " have ")
/tmp/ipykernel_20089/87036369.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r"\'re", " are ")
/tmp/ipykernel_20089/87036369.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r"\'d", " would ")
/tmp/ipykernel_20089/87036369.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r"\'ll", " will ")
/tmp/ipykernel_20089/87036369.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r"\'scuse", " excuse ")
/tmp/ipykernel_20089/87036369.py:20: FutureWarning: The default va

0.8549729504785685


In [14]:
df_val.head()

,Unnamed: 0,less_toxic,more_toxic
0,1,I read wrote; neither two operations includes ...,people read crap posted idiots like ?
1,2,""" I know """"personal attacks"""" referring to. An...",people read crap posted idiots like ?
2,5,"U POUR ADMIN, U UPDATE VANDAL COUNT, WHILE REA...",Your relentless edit warring Your messages alm...
3,9,"Failure follow guidelines WP:NOR, WP:RS, WP:Ve...",Against community know devils commmunity Jimbo...
4,10,Get encyclopedia nerds......(kanichiwa bitches),"AND TO THE PARTY WHO FUCKS ME, FUCK U ! I ' LL..."


In [15]:

df_sub = pd.read_csv('/home/xiaoguzai/数据/kaggle有毒评论对比/Jigsaw Rate Severity of Toxic Comments/comments_to_score.csv')
#df_sub['text'] = df_sub['text'].progress_apply(text_cleaning)
df_sub = clean(df_sub,'text')
X_test = vec.transform(df_sub['text'])
p3 = model.predict(X_test)
df_sub['score'] = p3
df_sub.to_csv("submission.csv", index=False)

/tmp/ipykernel_20089/87036369.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r"\'ve", " have ")
/tmp/ipykernel_20089/87036369.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r"\'re", " are ")
/tmp/ipykernel_20089/87036369.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r"\'d", " would ")
/tmp/ipykernel_20089/87036369.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r"\'ll", " will ")
/tmp/ipykernel_20089/87036369.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r"\'scuse", " excuse ")
/tmp/ipykernel_20089/87036369.py:20: FutureWarning: The default va